### 4. Use Autoencoder to implement anomaly detection. Build the model by using:

#### a. Import required libraries

In [28]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers, Model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
data = pd.read_csv("http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv", header = None)
data

,0,1,2,3,4,5,6,7,8,9,...,131,132,133,134,135,136,137,138,139,140
0,-0.112522,-2.827204,-3.773897,-4.349751,-4.376041,-3.474986,-2.181408,-1.818286,-1.250522,-0.477492,...,0.792168,0.933541,0.796958,0.578621,0.257740,0.228077,0.123431,0.925286,0.193137,1.0
1,-1.100878,-3.996840,-4.285843,-4.506579,-4.022377,-3.234368,-1.566126,-0.992258,-0.754680,0.042321,...,0.538356,0.656881,0.787490,0.724046,0.555784,0.476333,0.773820,1.119621,-1.436250,1.0
2,-0.567088,-2.593450,-3.874230,-4.584095,-4.187449,-3.151462,-1.742940,-1.490659,-1.183580,-0.394229,...,0.886073,0.531452,0.311377,-0.021919,-0.713683,-0.532197,0.321097,0.904227,-0.421797,1.0
3,0.490473,-1.914407,-3.616364,-4.318823,-4.268016,-3.881110,-2.993280,-1.671131,-1.333884,-0.965629,...,0.350816,0.499111,0.600345,0.842069,0.952074,0.990133,1.086798,1.403011,-0.383564,1.0
4,0.800232,-0.874252,-2.384761,-3.973292,-4.338224,-3.802422,-2.534510,-1.783423,-1.594450,-0.753199,...,1.148884,0.958434,1.059025,1.371682,1.277392,0.960304,0.971020,1.614392,1.421456,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,0.608558,-0.335651,-0.990948,-1.784153,-2.626145,-2.957065,-2.931897,-2.664816,-2.090137,-1.461841,...,1.757705,2.291923,2.704595,2.451519,2.017396,1.704358,1.688542,1.629593,1.342651,0.0
4994,-2.060402,-2.860116,-3.405074,-3.748719,-3.513561,-3.006545,-2.234850,-1.593270,-1.075279,-0.976047,...,1.388947,2.079675,2.433375,2.159484,1.819747,1.534767,1.696818,1.483832,1.047612,0.0
4995,-1.122969,-2.252925,-2.867628,-3.358605,-3.167849,-2.638360,-1.664162,-0.935655,-0.866953,-0.645363,...,-0.472419,-1.310147,-2.029521,-3.221294,-4.176790,-4.009720,-2.874136,-2.008369,-1.808334,0.0
4996,-0.547705,-1.889545,-2.839779,-3.457912,-3.929149,-3.966026,-3.492560,-2.695270,-1.849691,-1.374321,...,1.258419,1.907530,2.280888,1.895242,1.437702,1.193433,1.261335,1.150449,0.804932,0.0


In [5]:
data.shape

(4998, 141)

In [7]:
features = data.drop(140, axis=1)
target = data[140]
x_train, x_test, y_train, y_test = train_test_split( features, target, test_size=0.2, stratify=target)

In [33]:
class AutoEncoder(keras.Model):
    def __init__(self, output_units, ldim=8):
        super().__init__()
        self.encoder = keras.Sequential([
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dropout(0.1),
            keras.layers.Dense(32, activation='relu'),
            keras.layers.Dropout(0.1),
            keras.layers.Dense(16, activation='relu'),
            keras.layers.Dropout(0.1),
            keras.layers.Dense(ldim, activation='relu')
        ])
        self.decoder = keras.Sequential([
            keras.layers.Dense(16, activation='relu'),
            keras.layers.Dropout(0.1),
            keras.layers.Dense(32, activation='relu'),
            keras.layers.Dropout(0.1),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dropout(0.1),
            keras.layers.Dense(output_units, activation='relu')
        ])
    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

In [34]:
model = AutoEncoder(output_units=x_train.shape[1])

In [35]:
model.compile( loss = 'msle', metrics = ['mse'], optimizer='adam'
)

In [36]:
h = model.fit(x_train, x_train, epochs=10)

Epoch 1/10
125/125 [==============================] - 4s 5ms/step - loss: 0.0552 - mse: 0.8722
Epoch 2/10
125/125 [==============================] - 1s 4ms/step - loss: 0.0325 - mse: 0.7979
Epoch 3/10
125/125 [==============================] - 1s 5ms/step - loss: 0.0278 - mse: 0.7837
Epoch 4/10
125/125 [==============================] - 1s 5ms/step - loss: 0.0253 - mse: 0.7754
Epoch 5/10
125/125 [==============================] - 1s 5ms/step - loss: 0.0230 - mse: 0.7677
Epoch 6/10
125/125 [==============================] - 1s 6ms/step - loss: 0.0206 - mse: 0.7592
Epoch 7/10
125/125 [==============================] - 1s 5ms/step - loss: 0.0193 - mse: 0.7544
Epoch 8/10
125/125 [==============================] - 1s 6ms/step - loss: 0.0173 - mse: 0.7493
Epoch 9/10
125/125 [==============================] - 1s 5ms/step - loss: 0.0165 - mse: 0.7477
Epoch 10/10
125/125 [==============================] - 1s 5ms/step - loss: 0.0159 - mse: 0.7452


#### b. Upload / access the dataset

#### c. Encoder converts it into latent representation

#### d. Decoder networks convert it back to the original input

#### e. Compile the models with Optimizer, Loss, and Evaluation Metrics